# TP7 - Connexion à MongoDB sous `Python`

#### Analyse de Données Massives - Master 1ère année


In [83]:
import pymongo
import pandas
import pprint

con = pymongo.MongoClient()

def affiche(res):
    for r in res:
        pprint.pprint(r)

## Exercices

Nous allons utiliser la base de données `test`, dans laquelle se trouve la collection `restaurants`. Celle-ci est composée de plus de 25000 retaurants new-yorkais, avec des évaluations de ceux-ci.

### 1. Créer une connexion vers cette collection


In [84]:
db = con.test.restaurants

### 2. Déterminer le nombre exact de restaurants.

In [85]:
db.count()

25359

### 3. Lister les différents quartiers représentés (*borough*) ainsi que les différents types de  cuisine (*cuisine*).

In [86]:
db.distinct("borough")

['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island', 'Missing']

In [87]:
db.distinct("cuisine")

['Bakery',
 'Hamburgers',
 'American ',
 'Irish',
 'Jewish/Kosher',
 'Delicatessen',
 'Ice Cream, Gelato, Yogurt, Ices',
 'Chinese',
 'Chicken',
 'Caribbean',
 'Turkish',
 'Donuts',
 'Sandwiches/Salads/Mixed Buffet',
 'Bagels/Pretzels',
 'Continental',
 'Pizza',
 'Italian',
 'Steak',
 'Polish',
 'Latin (Cuban, Dominican, Puerto Rican, South & Central American)',
 'German',
 'French',
 'Pizza/Italian',
 'Mexican',
 'Spanish',
 'Café/Coffee/Tea',
 'Tex-Mex',
 'Pancakes/Waffles',
 'Soul Food',
 'Seafood',
 'Hotdogs',
 'Greek',
 'Not Listed/Not Applicable',
 'African',
 'Japanese',
 'Indian',
 'Armenian',
 'Thai',
 'Chinese/Cuban',
 'Mediterranean',
 'Korean',
 'Bottled beverages, including water, sodas, juices, etc.',
 'Russian',
 'Eastern European',
 'Middle Eastern',
 'Asian',
 'Ethiopian',
 'Vegetarian',
 'Barbecue',
 'Egyptian',
 'English',
 'Other',
 'Sandwiches',
 'Portuguese',
 'Indonesian',
 'Chinese/Japanese',
 'Filipino',
 'Juice, Smoothies, Fruit Salads',
 'Brazilian',
 'Afghan

### 4. Lister les noms des restaurants, ainsi que leur quartier, ne proposant que des soupes (*cuisine* égale à *soups*).

In [88]:
res = db.find({ "cuisine": "Soups" }, { "_id": 0, "name": 1, "borough": 1})
affiche(res)

{'borough': 'Manhattan', 'name': 'Wichcraft Express'}
{'borough': 'Staten Island', 'name': 'Original Soupman Of Staten Island'}
{'borough': 'Manhattan', 'name': 'The Original Soupman'}
{'borough': 'Manhattan', 'name': 'Peasant Stock'}


### 5. Donner les nombre de restaurants ayant eu au moins une fois le *grade* (dans *grades*) `Z`.

In [89]:
db.find({ "grades.grade": "Z"}).count()

1337

### 6. Calculer le nombre de restaurants pour chaque quartier

In [90]:
res = db.aggregate([ { "$group": { "_id": "$borough", "nb": { "$sum": 1 } } } ])
affiche(res)

{'_id': 'Missing', 'nb': 51}
{'_id': 'Staten Island', 'nb': 969}
{'_id': 'Manhattan', 'nb': 10259}
{'_id': 'Bronx', 'nb': 2338}
{'_id': 'Queens', 'nb': 5656}
{'_id': 'Brooklyn', 'nb': 6086}


### 7. Donner les cinq types de cuisine les plus présentes dans New-York (avec le plus de restaurants donc)

In [91]:
res = db.aggregate([ 
    { "$group": { "_id": "$cuisine" , "nb" : { "$sum" : 1 } } },
    { "$sort" : { "nb": - 1 } },
    { "$limit" : 5 }
])
affiche(res)

{'_id': 'American ', 'nb': 6183}
{'_id': 'Chinese', 'nb': 2418}
{'_id': 'Café/Coffee/Tea', 'nb': 1214}
{'_id': 'Pizza', 'nb': 1163}
{'_id': 'Italian', 'nb': 1069}


### 8. Pour chaque *grade* possible, donner le *score* moyen, minimum et maximum.

In [92]:
res = db.aggregate([
    { "$unwind" : "$grades"},
    { "$group" : { 
        "_id": "$grades.grade", 
        "nb" : { "$sum" : 1 },
        "moy" : { "$avg" : "$grades.score" },
        "min" : { "$min" : "$grades.score" },
        "max" : { "$max" : "$grades.score" }
    } }
])
affiche(res)

{'_id': 'P', 'max': 43, 'min': 0, 'moy': 5.506265664160401, 'nb': 1197}
{'_id': 'C', 'max': 131, 'min': -1, 'moy': 30.70079491255962, 'nb': 3145}
{'_id': 'Z', 'max': 71, 'min': -1, 'moy': 23.702318623784592, 'nb': 1337}
{'_id': 'Not Yet Graded', 'max': 75, 'min': 0, 'moy': 15.853515625, 'nb': 525}
{'_id': 'B', 'max': 57, 'min': -1, 'moy': 20.002459731809886, 'nb': 12603}
{'_id': 'A', 'max': 27, 'min': 0, 'moy': 9.021954029147022, 'nb': 74656}


### 9. Créer un `DataFrame` contenant les restaurants en ligne et en colonne le score moyen obtenu pour chaque *grade* possible. Si un restaurant n'a pas eu d'évaluation d'un *grade*, nous aurons une valeur absente.

In [93]:
res = db.aggregate([
    { "$group": { "_id": "$name", "nb": { "$sum": 1 } } },
    { "$sort": { "nb": -1 } },
    { "$limit": 10 }
])
affiche(res)

{'_id': 'Subway', 'nb': 421}
{'_id': 'Starbucks Coffee', 'nb': 223}
{'_id': "Mcdonald'S", 'nb': 208}
{'_id': "Dunkin' Donuts", 'nb': 206}
{'_id': 'Dunkin Donuts', 'nb': 200}
{'_id': '', 'nb': 151}
{'_id': 'Kennedy Fried Chicken', 'nb': 88}
{'_id': 'Crown Fried Chicken', 'nb': 81}
{'_id': 'Burger King', 'nb': 80}
{'_id': "Domino'S Pizza", 'nb': 67}


In [94]:
res = db.aggregate([
    { "$unwind" : "$grades"},
    { "$group" : { 
        "_id": { "num": "$restaurant_id", "grade": "$grades.grade" }, 
        "moy" : { "$avg" : "$grades.score" }
    } }
])
l = list(res)
l[:5]

[{'_id': {'grade': 'Not Yet Graded', 'num': '50018736'}, 'moy': 31.0},
 {'_id': {'grade': 'A', 'num': '50018705'}, 'moy': 2.0},
 {'_id': {'grade': 'Not Yet Graded', 'num': '50018704'}, 'moy': 34.0},
 {'_id': {'grade': 'Not Yet Graded', 'num': '50018661'}, 'moy': 2.0},
 {'_id': {'grade': 'Not Yet Graded', 'num': '50018547'}, 'moy': 29.0}]

In [95]:
df = pandas.DataFrame({
    "num"  : [r["_id"]["num"] for r in l],
    "grade": [r["_id"]["grade"] for r in l],
    "score": [r["moy"] for r in l]
})
df.head()

,grade,num,score
0,Not Yet Graded,50018736,31.0
1,A,50018705,2.0
2,Not Yet Graded,50018704,34.0
3,Not Yet Graded,50018661,2.0
4,Not Yet Graded,50018547,29.0


In [96]:
dfbis = df.pivot(index = "num", columns = "grade", values = "score")
dfbis.head()

grade,A,B,C,Not Yet Graded,P,Z
num,,,,,,
30075445,6.750000,14.0,NaN,NaN,NaN,NaN
30112340,10.666667,23.0,NaN,NaN,NaN,NaN
30191841,9.250000,NaN,NaN,NaN,NaN,NaN
40356018,9.000000,NaN,NaN,NaN,NaN,NaN
40356068,13.000000,25.0,NaN,NaN,NaN,20.0


In [97]:
dfbis.shape

(24621, 6)

In [98]:
dfbis.count()

grade
A                 23440
B                  8280
C                  2708
Not Yet Graded      512
P                  1154
Z                  1337
dtype: int64